In [1]:
print("Hello World")

Hello World


In [2]:
import pandas as pd
import glob
import os
import duckdb
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import numpy as np

Q1. Downloading the data <br>
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", <br>
we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023. <br>

Read the data for January. How many columns are there? <br>

* 16 <br>
* 17 <br>
* 18 <br>
* 19 <br>
* <span style="color:green"><b>20</b></span><br>

In [3]:
file_path='../data/yellow_tripdata_2023-01.parquet'
df_jan = duckdb.read_parquet(file_path)
duckdb.query(f"SELECT * FROM df_jan LIMIT 5").show()

┌──────────┬──────────────────────┬───────────────────────┬─────────────────┬───────────────┬────────────┬────────────────────┬──────────────┬──────────────┬──────────────┬─────────────┬────────┬─────────┬────────────┬──────────────┬───────────────────────┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datetime │ passenger_count │ trip_distance │ RatecodeID │ store_and_fwd_flag │ PULocationID │ DOLocationID │ payment_type │ fare_amount │ extra  │ mta_tax │ tip_amount │ tolls_amount │ improvement_surcharge │ total_amount │ congestion_surcharge │ airport_fee │
│  int64   │      timestamp       │       timestamp       │     double      │    double     │   double   │      varchar       │    int64     │    int64     │    int64     │   double    │ double │ double  │   double   │    double    │        double         │    double    │        double        │   double    │
├──────────┼──────────────────────┼───────────────────────┼───────────

Q2. Computing duration <br>
Now let's compute the duration variable. It should contain the duration of a ride in minutes. <br>
 
What's the standard deviation of the trips duration in January? <br>

* 32.59 <br>
* <span style="color:green"><b>42.59</b></span><br>
* 52.59 <br>
* 62.59 <br>


In [4]:
str_query = """ 
        SELECT 
            STDDEV(EXTRACT(
                        EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0) 
                AS stddev_trip_duration_minutes
        FROM  df_jan
        LIMIT 5 
"""
duckdb.query(str_query).show()

┌──────────────────────────────┐
│ stddev_trip_duration_minutes │
│            double            │
├──────────────────────────────┤
│           42.594351241955756 │
└──────────────────────────────┘



Q3. Dropping outliers <br>
Next, we need to check the distribution of the duration variable. <br>
There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 <br>  minutes (inclusive). <br>

What fraction of the records left after you dropped the outliers? <br>

* 90% <br>
* 92% <br>
* 95% <br>
* <span style="color:green"><b>98 %</b></span><br>

In [5]:
str_query = """ 

    SELECT (SUM( 
                CASE WHEN trip_duration_minutes BETWEEN 1.0 AND 60.0 THEN 1
                ELSE 0 
                END
           )*100)/COUNT(*) AS trip_duration_minutes_normal 
        FROM (

    SELECT EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0
                  AS trip_duration_minutes
        FROM df_jan
            ) 
"""
duckdb.query(str_query).show()
df_outlier_removed = duckdb.query(str_query)

┌──────────────────────────────┐
│ trip_duration_minutes_normal │
│            double            │
├──────────────────────────────┤
│             98.1220282212598 │
└──────────────────────────────┘



Q4. One-hot encoding <br>
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our <br> model. <br>

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will <br> label encode them) <br>
Fit a dictionary vectorizer <br>
Get a feature matrix from it <br>
What's the dimensionality of this matrix (number of columns)? <br>

* 2 <br>
* 155 <br>
* 345 <br>
* <span style="color:green"><b>515</b></span><br>
* 715 <br>

In [6]:
str_query = """
        SELECT CAST(PULocationID  AS STRING) as PULocationID,
                CAST(DOLocationID  as STRING) as DOLocationID
        FROM df_jan
                WHERE EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0 
        BETWEEN 1.0 AND 60.0        
"""

df_one_hot = duckdb.sql(str_query).df()
df_one_hot.shape
list_df_jan = df_one_hot.to_dict(orient='records')
vec = DictVectorizer() 
mat_jan = vec.fit_transform(list_df_jan)
print("Shape of Transformed Data: ({}, {})".format(*mat_jan.shape)) 


Shape of Transformed Data: (3009173, 515)


## Q5. Training a model <br>

Now let's use the feature matrix from the previous step to train a model.  <br>

* Train a plain linear regression model with default parameters, where duration is the response variable <br>
* Calculate the RMSE of the model on the training data <br>

What's the RMSE on train? <br>

* 3.64 <br>
* <span style="color:green"><b>7.64</b></span><br>
* 11.64 <br>
* 16.64 <br>

In [7]:
str_query = """ 
        SELECT 
            EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0 
                AS trip_duration_minutes
        FROM  df_jan
        WHERE EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0 
                BETWEEN 1.0 AND 60.0  
"""

y = duckdb.sql(str_query).df().to_numpy()
X = mat_jan


In [8]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [9]:
y_pred = model.predict(X)

In [10]:
rmse = root_mean_squared_error(y, y_pred)
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 7.649261927011587


## Q6. Evaluating the model <br>

Now let's apply this model to the validation dataset (February 2023). <br>

What's the RMSE on validation? <br>

* 3.81 <br>
* <span style="color:green"><b>7.81</b></span><br>
* 11.81 <br>
* 16.81 <br>

In [11]:
## Read in the data 
file_path='../data/yellow_tripdata_2023-02.parquet'
df_feb = duckdb.read_parquet(file_path).df()


## Remove Outliers 
str_query = """
        SELECT CAST(PULocationID  AS STRING) as PULocationID,
                CAST(DOLocationID  as STRING) as DOLocationID
        FROM df_feb
                WHERE EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0 
        BETWEEN 1.0 AND 60.0        
"""

## One Hot Encode 
df_one_hot = duckdb.sql(str_query).df()
df_one_hot.shape
list_df_feb = df_one_hot.to_dict(orient='records')
vec = DictVectorizer() 
mat_feb = vec.fit_transform(list_df_feb)


## Prepare train data 
str_query = """ 
        SELECT 
            EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0 
                AS trip_duration_minutes
        FROM  df_feb
        WHERE EXTRACT(EPOCH FROM (tpep_dropoff_datetime - tpep_pickup_datetime))/60.0 
                BETWEEN 1.0 AND 60.0  
"""

y = duckdb.sql(str_query).df().to_numpy()
X = mat_feb 

## Train an predict model 
model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

rmse = root_mean_squared_error(y, y_pred)
print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 7.778948594646286
